In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Script to CSV Process

1. Load script `.txt` file.  Use [PDF Extractor](https://pdfextractor.com/) to convert any `.pdf`s to `.txt`s.
2. Remove garbage rows
3. Inspect rows removed
4. Format lines
5. Create `uppercase_unique_form.csv`
6. Manually fill out `uppercase_unique_form.csv` columns `is_character_name` and `is_dialogue` in Excel or other `.csv` editor.
7. Load `uppercase_unique.csv` and filter out rows
8. Inspect rows removed
9. Merge lines into best guesses for columns "character" and "line"
10. Manually clean the "line" column by removing text that is not dialogue.  There will be a lot, so nearly every cell will have to be cleaned!


See the Guardians of the Galaxy script at the end of this notebook for an example of this process.

## Remove Garbage Rows

In [2]:
def remove_regex_rows(df,string):
    rows_to_remove = df["line"].str.contains(string,case=True,regex=True)
    print("Removed",rows_to_remove.sum(),"rows that match regex \"" + string + "\"")
    
    return df[~rows_to_remove], df[rows_to_remove]

def remove_whitespace_rows(df):
    whitespace_rows = df["line"].str.isspace()
    print(whitespace_rows.sum(),"whitespace rows removed")
    return df[~whitespace_rows], df[whitespace_rows]

def remove_movie_text_rows(df):
    
    regex_strings = ["^\[[^\[\]]*\]$"]
    
    removed_rows = pd.DataFrame()
    
    for regex_string in regex_strings:
        df, new_rows_to_remove = remove_regex_rows(df,regex_string)
        
        removed_rows = pd.concat([removed_rows, new_rows_to_remove])
    
    return df, removed_rows


def remove_page_number_rows(df):
    page_rows = df["line"].str.isnumeric()
    print(page_rows.sum(),"page number rows removed")
    
    return df[~page_rows], df[page_rows]

def remove_garbage_rows(df):
    df, whitespace_rows = remove_whitespace_rows(df)
    df, page_number_rows = remove_page_number_rows(df)
    df, movie_text_rows = remove_movie_text_rows(df)
    
    garbage_rows = pd.concat([whitespace_rows, page_number_rows, movie_text_rows], 
                            keys = ["whitespace","page_numbers","movie_text"])
    
    print("-----------------------------------------")
    print(garbage_rows.shape[0],"total rows removed\n")
    
    return df, garbage_rows

## Format Character Lines

In [3]:
def remove_regex(df,string):
    print("Removed",df["line"].str.count(string).sum(),"occurences of regex \"" + string + "\"")
    df["line"] = df["line"].str.replace(string,"",case=True,regex=True)

def remove_leading_trailing_whitespace(df):
    df["line"] = df["line"].str.replace("[ \t]+$","")
    df["line"] = df["line"].str.replace("^[ \t]+","")

def format_lines(df):
    remove_leading_trailing_whitespace(df)
    
    regex_strings = ["\[[^\[\]]*\]"]
    
    for regex_string in regex_strings:
        remove_regex(df, regex_string)
    
    print("")
    
    remove_leading_trailing_whitespace(df)

## Create Character / Line Columns

In [4]:
def create_character_line_columns(df):
    character = df.line.str.extract(r'(^[^:]*:)')[0]
    line = df.line.str.extract(r'(:.*$)')[0]
    
    character = character.str.replace(":$","",case=True,regex=True)
    line = line.str.replace("^[ \t]*:[ \t]*","",case=True,regex=True)
    
    character = character.str.upper()
    
    return pd.DataFrame({"character": character, "line": line})

## Avengers: Infinity War

In [5]:
infinity_war = pd.read_csv("./script txts/infinity_war.txt",sep="\n",header=None,names=["line"])

print(infinity_war.shape)

infinity_war, garbage_rows = remove_garbage_rows(infinity_war)
format_lines(infinity_war)

#print(infinity_war.shape)
#print(infinity_war[~infinity_war["line"].str.contains("^[^:]*:", regex=True)].shape)
#infinity_war[~infinity_war["line"].str.contains("^[^:]*:", regex=True)]


infinity_war = create_character_line_columns(infinity_war)

infinity_war.to_csv("./uncleaned/infinity_war_uncleaned.csv", index=False)

infinity_war = pd.read_csv("./cleaned/infinity_war.csv")

infinity_war.head()

(1317, 1)
0 whitespace rows removed
0 page number rows removed
Removed 315 rows that match regex "^\[[^\[\]]*\]$"
-----------------------------------------
315 total rows removed

Removed 460 occurences of regex "\[[^\[\]]*\]"



,character,line
0,ASGARDIAN PA,"This is the Asgardian refugee vessel Statesman. We are under assault, I repeat, we are under assault - The engines are dead, life support failing. Requesting aid from any vessel within range. We are 22 jump points out of Asgard. Our crew is made up of Asgardian families, we have very few soldiers here. This is not a warcraft. I repeat, this is not a warcraft!"
1,EBONY MAW,"Hear me, and rejoice. You have had the privilege of being saved by the Great Titan.... You may think this is suffering... no. It is salvation. Universal scales tip toward balance because of your sacrifice. Smile... for even in death, you have become Children of Thanos."
2,THANOS,"I know what it's like to lose. To feel so desperately that you're right... yet to fail, nonetheless. It's frightening. Turns the legs to jelly. I ask you, to what end? Dread it. Run from it. Destiny arrives all the same. And now, it's here. Or should I say... I AM."
3,THOR,You talk too much.
4,THANOS,"The Tesseract, or your brother's head. I assume you have a preference."
